# trcFormat Converter
### Johanna Rahm, Sebastian Malkusch
#### Research group Heilemann
#### Institute for Physical and Theoretical Chemistry, Goethe University Frankfurt a.M.
Convert thunderSTORM .csv results table or rapidSTORM localization .txt file to .trc file (seg_id, image_number, x-position [pixel], placeholder, y-position [pixel], intensity). <br/>
The thunderSTORM x/y-positions have to be in nm, intensity in ADU. <br/>
The .trc file is needed for the trackAnalysis JNB and Hidden Markov Modeling. 

## Choose a localization software

In [1]:
from pySPT.widgets import widgetDirectoryStructure
from pySPT.widgets import widgetTrcFormat
from pySPT.analysis import trcFormat
from pySPT.widgets import widgetColumnSort
widget_trc_format = widgetTrcFormat.WidgetTrcFormat()
display(widget_trc_format.software_button)

RadioButtons(options=('thunderSTORM', 'rapidSTORM'), value='thunderSTORM')

## Load file

thunderSTORM: Insert the full file path of the tracked thunderSTORM .csv file or browse for it (Example path: F:\Marburg\single_colour_tracking\resting\160404_CS5_Cells\160404_CS5_Cell4\cell04.tracked.loc.csv) <br/>
rapidSTORM: Insert the full file path of the tracked rapidSTORM .txt file or browse for it. (Example path: F:\Marburg\single_colour_tracking\resting\160404_CS5_Cell1\cell_1_MMStack_Pos0.ome.tif.tracked.loc.txt)

In [2]:
widget_dir_structure = widgetDirectoryStructure.WidgetDirStructure()
trc_format = trcFormat.TrcFormat()
display(widget_trc_format.file_text_box, widget_trc_format.file_button)
widget_trc_format.file_button.on_click(widget_trc_format.open_file)
widget_trc_format.file_text_box.observe(widget_trc_format.change_file_box)

Text(value='path', description='Complete path', placeholder='Type something', style=DescriptionStyle(descripti…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

## Run analysis

A new file with columns: seg_id, frame, x, y, placeholder, intensity will be created.

In [3]:
display(widget_trc_format.pixel_size_box, widget_trc_format.min_track_length_box)

Text(value='158', description='pixel size in nm', placeholder='Type something', style=DescriptionStyle(descrip…

Text(value='2', description='Min track length', placeholder='Type something', style=DescriptionStyle(descripti…

In [4]:
display(widget_trc_format.run_button)
def run_analysis(event):
    widget_trc_format.create_clear_output()
    display(widget_trc_format.run_button)
    if widget_trc_format.got_file_name:
        if widget_trc_format.software_button.value == "thunderSTORM":
            widget_column_sort = widgetColumnSort.WidgetColumnSort(widget_trc_format.file_text_box.value, widget_trc_format.software_button.value, ['"track_id"' , '"x [nm]"', '"y [nm]"', '"frame"', '"intensity [photon]"', '"seg_id"'])
        elif widget_trc_format.software_button.value == "rapidSTORM":
            widget_column_sort = widgetColumnSort.WidgetColumnSort(widget_trc_format.file_text_box.value, widget_trc_format.software_button.value, ['"Position-0-0"', '"Position-1-0"', '"Amplitude-0-0"', '"ImageNumber-0-0"', '"seg_id"', '"track_id"'])
        widget_column_sort.check_header()
        if widget_column_sort.correct_header:
            widget_column_sort.run_column_sort()
            trc_format.software = widget_trc_format.software_button.value
            trc_format.column_order = widget_column_sort.column_order
            trc_format.pixel_size = widget_trc_format.pixel_size_box.value
            trc_format.min_track_length = widget_trc_format.min_track_length_box.value
            trc_format.file_name = widget_trc_format.file_text_box.value
            trc_format.load_localization_file()
        else:
            widget_trc_format.warning_wrong_file()
    else:
        widget_trc_format.warning_wrong_file_path()
widget_trc_format.run_button.on_click(run_analysis)

Button(description='run', style=ButtonStyle(), tooltip='run the analysis')

Conversion successful.


## Save analysis

The .trc file will be saved in the pySPT/analysis folder.

In [5]:
display(widget_trc_format.save_button)
def save_analysis(event):
    widget_trc_format.create_clear_output()
    display(widget_trc_format.save_button)
    #widget_dir_structure.name_handling(widget_trc_format.file_name)
    widget_dir_structure.name_handling(widget_trc_format.file_text_box.value)
    widget_dir_structure.create_raw_base_name()
    widget_dir_structure.sub_folder = "\\analysis"
    widget_dir_structure.create_folder()
    trc_format.save_trc_file_analysis(widget_dir_structure.sub_folder_dir, widget_dir_structure.raw_base_name)
    widget_dir_structure.sub_folder = "\\hmm"
    widget_dir_structure.create_folder()
    trc_format.save_trc_file_hmm(widget_dir_structure.sub_folder_dir, widget_dir_structure.raw_base_name)
widget_trc_format.save_button.on_click(save_analysis)

Button(description='save', style=ButtonStyle(), tooltip='save the results')